In [20]:
from dask_jobqueue import SLURMCluster
client.restart()
#cluster = SLURMCluster() #add arguments if want a cluster outside of the set up defult
cluster = SLURMCluster(cores=24, processes=6, walltime="00:15:00", queue="compute", memory= "120GB")

/home/caolinnh/miniconda3/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36741 instead
  http_address["port"], self.http_server.port


In [21]:
cluster.scale(jobs=2) #scale to however many nodes wanted

In [29]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://10.22.254.91:34278 Dashboard: http://10.22.254.91:36741/status,Cluster Workers: 12 Cores: 48 Memory: 240.00 GB


In [30]:
client

Client Scheduler: tcp://10.22.254.91:34278 Dashboard: http://10.22.254.91:36741/status,Cluster Workers: 12 Cores: 48 Memory: 240.00 GB


In [31]:
from time import time, sleep

from distributed import Client, LocalCluster, as_completed
import pandas as pd
from time import time
import os
import os.path
import glob

import os
import dask
import dask.dataframe as dd  # Important line

In [32]:

experiment_runs = 1  # Number of times to repeat r/w tests

#sourceDir = '/oasis/scratch/comet/dougfe/temp_project/df_benchmarking/daskBench/benchHome/'  # Location of outputs and source csv files. Copy output of $pwd if unsure of what to put here

sourceDir = '/oasis/scratch/comet/caolinnh/temp_project/benchHome/'

df_fileprefix = 'bench'  # Central name for all files produced during benchmark, includes '.' for later attachment to extensions in extensionList

df_source = ["2500mb.csv", "5000mb.csv", "12500mb.csv"] # List of dataframes to run analysis on, will help name folder


def write_bench(df_in, filename, reportPath):
    for i in range(experiment_runs):
        reportFile = open(reportPath, 'a')
        savetime = time()
        dd.to_csv(df_in, filename)
        reportFile.write(str(time()-savetime)+' ')
        reportFile.close()

def read_bench(readSource, reportPath):
    reportFile = open(reportPath, 'a')
    for i in range(experiment_runs):
        savetime = time()
        tempDF = dd.read_csv(readSource, dtype = 'object', low_memory=False, error_bad_lines=False, header=None)
        #tempDF = dd.read_csv(readSource, dtype = 'object', low_memory = False)
        tempDF = client.persist(tempDF)
        tempDF = tempDF.compute()
        #tempDF = tempDF.compute()
        reportFile.write(str(time()-savetime)+' ')
        tempDF = ''
    reportFile.close()

def writeParquetBench(df_in, filename, reportPath):
    for i in range(experiment_runs):
        #if os.path.exists(filename):
        #    os.remove(filename)
        reportFile = open(reportPath, 'a')
        savetime = time()
        dd.to_parquet(df_in, filename)
        reportFile.write(str(time()-savetime)+' ')
        reportFile.close()

def readParquetBench(readSource, reportPath):
    reportFile = open(reportPath, 'a')
    for i in range(experiment_runs):
        savetime = time()
        tempDF = dd.read_parquet(readSource, dtype = 'object', low_memory = False)
        tempDF = tempDF.persist()
        reportFile.write(str(time()-savetime)+' ')
        tempDF = ''
    reportFile.close()

In [33]:
def dug ():
    extension = 'csv'
    for sourceCSV in df_source:   

        testFilepath = sourceDir + df_fileprefix + sourceCSV[:-3] + extension  # Name+loc of output files
        reportW = sourceDir + extension + '_w_' + sourceCSV[:-4] + '.txt'
        currentDF = dd.read_csv(sourceDir+sourceCSV, dtype = 'object', low_memory = False)
        currentDF
        write_bench(currentDF, testFilepath, reportW)

        reportR = sourceDir + extension + '_r_' + sourceCSV[:-4] + '.txt'
        read_bench(sourceDir+sourceCSV, reportR)
        
    
    extension = 'parquet'
    for sourceCSV in df_source:   

        testFilepath = sourceDir + df_fileprefix + sourceCSV[:-3] + extension  # Name+loc of output files
        reportW = sourceDir + extension + '_w_' + sourceCSV[:-4] + '.txt'
        currentDF = dd.read_csv(sourceDir+sourceCSV, dtype = 'object', low_memory = False)
        writeParquetBench(currentDF, testFilepath, reportW)

        reportR = sourceDir + extension + '_r_' + sourceCSV[:-4] + '.txt'
        readParquetBench(sourceDir+sourceCSV[:-3]+'parquet', reportR)
    return 'Benchmark complete.'

In [34]:
%%time 

if __name__ == "__main__":
    for x in range(1):
        extension = 'csv'
        for sourceCSV in df_source:   

            testFilepath = sourceDir + df_fileprefix + sourceCSV[:-3] + extension  # Name+loc of output files
            reportW = sourceDir + extension + '_w_' + "3node" + sourceCSV[:-4] + '.txt'
            currentDF = dd.read_csv(sourceDir+sourceCSV, dtype = 'object', low_memory=False, error_bad_lines=False, header=None)
            currentDF
            write_bench(currentDF, testFilepath, reportW)

            reportR = sourceDir + extension + '_r_' + "3node" + sourceCSV[:-4] + '.txt'
            read_bench(sourceDir+sourceCSV, reportR)
            #currentDF = currentDF.compute()
        print(x)
            


        #client.upload_file("/oasis/scratch/comet/caolinnh/temp_project/test.py")

        #future = client.submit(dug)
        #future = client.submit(exec(open("test.py").read())
        #future.result()

        #open("/oasis/scratch/comet/caolinnh/temp_project/1WBenchComet.py")).read()

0
CPU times: user 3min 28s, sys: 2min 17s, total: 5min 45s
Wall time: 10min 15s


In [ ]:
%%time
currentDF = currentDF.compute()

In [ ]:
%%time
df_source = ['25gb.csv'] # List of dataframes to run analysis on, will help name folder
if __name__ == "__main__":
    for i in range(1):

        extension = 'csv'
        for sourceCSV in df_source:   

            testFilepath = sourceDir + df_fileprefix + sourceCSV[:-3] + extension  # Name+loc of output files
            reportW = sourceDir + extension + '_w_' + "12core" + sourceCSV[:-4] + '.txt'
            #currentDF = dd.read_csv(sourceDir+sourceCSV, dtype = 'object', low_memory=False, error_bad_lines=False, header=None)
            currentDF
            

            reportR = sourceDir + extension + '_r_' + "12core" + sourceCSV[:-4] + '.txt'
            read_bench(sourceDir+sourceCSV, reportR)
            
            write_bench(tempDF, testFilepath, reportW)

In [51]:
client.persist(currentDF)

CPU times: user 150 ms, sys: 0 ns, total: 150 ms
Wall time: 149 ms


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75
npartitions=79,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [13]:
%%time
currentDF.head()

CPU times: user 26.4 ms, sys: 4.03 ms, total: 30.5 ms
Wall time: 2.46 s


,Unnamed: 0,JobIDRaw,step,JobID,UID,GID,AssocID,Cluster,JobName,User,...,MaxRSSTask,AveVMSize,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,ConsumedEnergy,ConsumedEnergyRaw,Layout,Comment,NodeList
0,0,33768591,batch,33768591.batch,NaN,NaN,7461,comet,batch,NaN,...,0.0,1758260K,1758260K,comet-11-19,0.0,0.0,0.0,Unknown,NaN,comet-11-19
1,1,33768607,batch,33768607.batch,NaN,NaN,7461,comet,batch,NaN,...,0.0,1758272K,1758272K,comet-05-35,0.0,0.0,0.0,Unknown,NaN,comet-05-35
2,2,33863502,batch,33863502.batch,NaN,NaN,7390,comet,batch,NaN,...,0.0,486392K,486392K,comet-07-67,0.0,0.0,0.0,Unknown,NaN,comet-07-67
3,3,33863502,0,33863502.0,NaN,NaN,7390,comet,hydra_pmi_proxy,NaN,...,0.0,30838930K,37647712K,comet-07-72,1.0,0.0,0.0,Cyclic,NaN,"comet-07-[67,72]"
4,4,33863503,batch,33863503.batch,NaN,NaN,7390,comet,batch,NaN,...,0.0,486388K,486388K,comet-22-65,0.0,0.0,0.0,Unknown,NaN,comet-22-65


In [25]:
def easy():
    sleep(1)
    print("easy")
    return "easy"

def hard():
    sleep(5)
    print("hard")
    return "hard"

def tast(x):
    sleep(x)
    print(f'task took: {x} seconds' )
    return(x)

In [27]:
if __name__ == "__main__":
       
        
        futures = []
        f1 = client.submit(easy) # key = f"{easy.__name__}{i}")
        for i in range(3):
            f1 = client.submit(easy) # key = f"{easy.__name__}{i}")
            futures.append(f1)
            f2 = client.submit(hard) # key = f"{hard.__name__}{i}")
            futures.append(f2)

        completed = as_completed(futures)
        
        for i in completed:
            i.result()

        future = client.submit(hard)
        future2 = client.submit(easy)
        print(future2.result())

easy
